In [1]:
import pandas as pd

from lib.utils import rv

## First lets check the 65min aggs

In [2]:
aggs = pd.read_csv("/Users/beneverman/Documents/Coding/QuantHive/IDVF-Oxford-v1/data/processed-5yr-93-minute/65min.csv", index_col=0)

In [3]:
aggs

,datetime,TMO,ABT,HD,MCD,PG,CAT,DIS,CCI,JNJ,...,COST,IBM,ADP,AVGO,WMT,AMGN,INTU,AXP,MMC,CB
0,2018-10-11 09:30:00-04:00,231.4025,68.9400,195.260,165.720,80.680,143.9900,112.295,107.9300,136.72,...,223.6400,142.6300,140.5400,235.2900,95.3400,196.330,206.100,103.690,81.180,130.04
1,2018-10-11 10:35:00-04:00,228.4800,68.6200,193.320,164.405,80.185,142.7600,111.960,107.0100,135.64,...,222.7690,141.5762,139.3662,232.7200,95.0200,195.780,204.730,103.000,80.960,128.15
2,2018-10-11 11:40:00-04:00,231.4800,69.3857,194.630,164.970,79.970,143.6000,112.580,107.0700,136.26,...,225.2500,142.2400,140.4100,233.8500,95.1000,197.770,205.390,103.660,81.130,128.42
3,2018-10-11 12:45:00-04:00,228.9800,68.8200,193.570,164.150,79.660,142.8800,112.190,106.7900,135.87,...,224.3500,141.3201,139.6500,232.5700,94.9400,195.720,203.990,103.330,80.870,127.92
4,2018-10-11 13:50:00-04:00,226.8900,68.3500,190.040,162.850,79.210,141.8500,111.200,105.3590,134.41,...,222.2557,139.9300,138.4400,229.4200,94.1500,194.650,201.700,102.280,79.980,126.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7511,2023-10-09 10:35:00-04:00,492.3400,96.0800,291.235,248.350,141.960,267.2700,83.895,92.6000,157.94,...,552.4000,141.2300,248.2800,842.9400,153.9400,271.065,529.726,149.035,192.720,210.19
7512,2023-10-09 11:40:00-04:00,492.7300,95.8850,292.350,247.935,142.340,267.8525,83.990,92.7200,157.58,...,554.6400,141.0100,248.2700,845.7300,154.1828,269.490,530.270,148.530,192.580,209.00
7513,2023-10-09 12:45:00-04:00,494.6027,96.4200,294.245,249.410,142.935,270.1700,84.515,93.3179,158.23,...,556.8100,141.7500,249.3700,853.2500,154.8300,271.390,533.900,149.050,193.005,209.89
7514,2023-10-09 13:50:00-04:00,497.1050,96.7100,295.300,249.300,143.010,271.0400,84.735,93.3600,158.35,...,558.5200,142.0152,250.0100,854.1507,155.4100,271.590,534.700,149.255,193.440,210.18


In [4]:
aggs.ffill(inplace=True)
aggs.bfill(inplace=True)
assert aggs.isna().sum().sum() == 0

In [5]:
aggs['datetime'] = pd.to_datetime(aggs['datetime'], utc=True).dt.tz_convert('US/Eastern')
aggs.set_index('datetime', inplace=True)

namelist = aggs.columns.tolist()

## Get RV (log vol)

In [6]:
window_length = 6*250
for ind in namelist:
    aggs[ind + "_logvol"] = rv(aggs[ind], window_length)

In [20]:
aggs.filter(regex='_logvol$').describe().T.describe()

,count,mean,std,min,25%,50%,75%,max
count,93.0,93.000000,93.000000,93.000000,93.000000,93.000000,93.000000,93.000000
mean,6016.0,-2.441154,0.443230,-3.237274,-2.768809,-2.496179,-2.103305,-1.684732
std,0.0,0.412830,0.097663,0.446434,0.446902,0.445635,0.399816,0.415971
min,6016.0,-3.259830,0.254324,-4.110838,-3.689441,-3.433807,-2.886104,-2.701530
25%,6016.0,-2.676796,0.370377,-3.489981,-3.051887,-2.778725,-2.342331,-1.944092
50%,6016.0,-2.432516,0.442391,-3.280757,-2.797995,-2.484664,-2.129733,-1.701497
75%,6016.0,-2.186705,0.504355,-2.926913,-2.474109,-2.216629,-1.824168,-1.437832
max,6016.0,-1.370248,0.668735,-1.993542,-1.697438,-1.245309,-0.950598,-0.371212


In [21]:
aggs.filter(regex='_logvol$').mean(axis=0).describe()

count    93.000000
mean     -2.441154
std       0.412830
min      -3.259830
25%      -2.676796
50%      -2.432516
75%      -2.186705
max      -1.370248
dtype: float64

## Get Raw Results

In [23]:
base_path = "/Users/beneverman/Documents/Coding/QuantHive/IDVF-Oxford-v1/outputs/2023-10-14_13-57-25"
result = pd.read_csv(f"{base_path}/result.csv", index_col=0)
report = pd.read_csv(f"{base_path}/report.csv", index_col=0)

In [27]:
result.describe().T.describe()

,count,mean,std,min,25%,50%,75%,max
count,186.0,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000
mean,4938.0,-2.417285,0.406468,-3.036241,-2.733475,-2.482199,-2.252076,-1.685586
std,0.0,0.427878,0.089630,0.441708,0.453684,0.450330,0.475122,0.413292
min,4938.0,-3.278718,0.211118,-4.039987,-3.728146,-3.460839,-3.305536,-2.701530
25%,4938.0,-2.682292,0.335261,-3.322059,-3.019254,-2.784453,-2.581922,-1.943349
50%,4938.0,-2.421819,0.409937,-3.042473,-2.713655,-2.476375,-2.255086,-1.701283
75%,4938.0,-2.127583,0.459879,-2.726436,-2.440555,-2.201689,-1.968370,-1.437560
max,4938.0,-1.330730,0.633257,-1.951062,-1.652885,-1.205718,-0.995173,-0.371139


In [31]:
report

,MSE,r2_score,MAPE
TMO,0.000011,0.999790,0.000497
ABT,0.000018,0.999875,0.000462
HD,0.000014,0.999917,0.000457
MCD,0.000024,0.999940,0.000417
PG,0.000014,0.999933,0.000376
...,...,...,...
AMGN,0.000021,0.999840,0.000470
INTU,0.000016,0.999826,0.000580
AXP,0.000047,0.999741,0.000667
MMC,0.000013,0.999937,0.000410


In [34]:
base_path2 = "/Users/beneverman/Documents/Coding/QuantHive/IDVF-Oxford-v1/outputs/2023-10-14_14-05-17"
result2 = pd.read_csv(f"{base_path}/result.csv", index_col=0)
report2 = pd.read_csv(f"{base_path}/report.csv", index_col=0)

In [39]:
report2.mean().to_dict()

{'MSE': 2.7828444984429477e-05,
 'r2_score': 0.9998002148665266,
 'MAPE': 0.0005433835400242786}